In [60]:
import git
repo_path = git.Repo('.', search_parent_directories=True).working_tree_dir

import numpy as np
import pandas as pd
import json
import os

In [61]:
folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQE\OptimizeHyperparams\Tol")
potential = 'AHO'
cutoff = 16

dataframes = {}

folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots = []
tolerance = []
abs_tolerance = []
strategy = []
popsize = []
maxiter = []


for f in folders:

    data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    num_vqe.append(data['num_VQE'])
    shots.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    tolerance.append(optimzer['tolerance'])

    success = sum(data['success'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))
    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))
    median_e.append(np.median(data['results']))
    delta_median_e.append(abs(exact_min_e - np.median(data['results'])))
    
    time_taken.append(data['total_VQE_time'])

    

In [62]:
df = pd.DataFrame({
    'folder': folders,
    #'num_vqe': num_vqe,
    #'shots': shots,
    'tolerance': tolerance,
    'Converged Runs': [f"{c}/100" for c in converged_runs],
    #'maxiter': maxiter,
    'Mean Iter': mean_iters,
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Time': time_taken
})

In [63]:
df[['Delta min E', 'Delta median E']] = df[['Delta min E', 'Delta median E']].astype(float)
df = df.sort_values('Delta median E', ascending=False)
df.to_excel(os.path.join(folder_path, "table.xlsx"), index=False)
df

,folder,tolerance,Converged Runs,Mean Iter,Delta min E,Delta median E,Exact,Time
0,0.0001,1.000000e-04,100/100,54,49.9791,13.34930,-1.16698e-03,0:13:23.954708
9,1e-10,1.000000e-10,100/100,55,63.9837,12.85720,-1.16698e-03,0:13:02.550372
5,1e-06,1.000000e-06,100/100,54,51.9563,12.72880,-1.16698e-03,0:13:30.419274
1,0.001,1.000000e-03,100/100,54,46.5894,12.19800,-1.16698e-03,0:13:41.419995
10,1e-11,1.000000e-11,100/100,54,69.9622,11.94930,-1.16698e-03,0:12:55.175903
6,1e-07,1.000000e-07,100/100,54,42.7684,11.69690,-1.16698e-03,0:13:38.377151
3,0.1,1.000000e-01,100/100,54,49.7489,11.40020,-1.16698e-03,0:13:16.288341
7,1e-08,1.000000e-08,100/100,55,42.4446,11.20530,-1.16698e-03,0:13:24.222859
8,1e-09,1.000000e-09,100/100,54,50.1210,10.06150,-1.16698e-03,0:13:35.232975
2,0.01,1.000000e-02,100/100,53,37.9392,9.86355,-1.16698e-03,0:13:11.088455
